# Observing sheet for visit to Tammin in January 2019

This notebook uses `astropy`, `astroplan`, and `jinja2` templates to convert a CSV list of targets generated with [DSO Browser](https://dso-browser.com) into a nice printable observing template. The resulting template can be [found here](targets.html).

This notebook is annotated to explain how the code works

## Load dependencies

Loads required python modules.

In [1]:
import warnings
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, Angle
from astroplan import Observer
from dateutil.parser import parse as parse_date
import numpy as np
import pandas

# This comes from the folder with the name astromateur in the same folder as
# this notebook. I wrote this code to start collecting stuff I plan to use
# again later
from astromateur.pocketatlas import map_number
from astromateur.render import template

## Utility functions

These are utility functions used throughout the later code. Each function is documented with a brief description.

In [2]:
def parse_to_Time(x):
    '''Take a date and time as a string and convert it to an astropy.Time
    object. Uses dateutil.parser.parse_date and therefore accepts anything
    it does.'''
    return Time(
        parse_date(x).timestamp(),
        format='unix'
    )


def altitude_at(target, time, observer):
    '''Find the altitude of the target for the given observer at the given
    time'''
    return observer.altaz(time, target).alt


def subset_and_rename(x, columns):
    '''Convenience combination of pandas DataFrame subset and rename'''
    return x[list(columns.keys())].rename(columns=columns)


def safe_astropy_time_to_datetime(xs, observer):
    '''Convert astropy.Time into datetime objects, returning None for
    the special values corresponding to something never happening'''
    return [
        observer.astropy_time_to_datetime(x)
        if x.jd > -990
        else None
        for x in xs
    ]

## Load the data

Loads the data from the input csv in [targets.csv](targets.csv). Performs some preliminary clean-up of the file, and drops duplicate records. Show the first five rows.

In [3]:
data = pandas.read_csv('targets.csv') \
    .replace(np.nan, '', regex=True) \
    .replace('º', '°', regex=True) \
    .drop_duplicates('Catalogue Entry')

data.head(5)

,Catalogue Entry,Familiar Name,Alternative Entries,Type,Constellation,Right Ascension,Declination,Magnitude,Size,Surface Brightness,Rise time over 10º,Transit Time,Set time below 10º,Maximum Elevation
0,M 79,,"NGC 1904, ESO 487-SC7, GCL 10",Globular cluster,Lepus,"05h 24' 11""","-24° 31' 25""",7.7,9.6',21.24,3:36pm,9:52pm,4:08am,81°
1,M 38,,"NGC 1912, OCL 433",Open cluster,Auriga,"05h 28' 43""","35° 51' 18""",6.4,15',20.91,7:07pm,9:56pm,0:46am,21°
2,M 1,Crab Nebula,"NGC 1952, Sh 2-244",Planetary nebula,Taurus,"05h 34' 30""","22° 00' 60""",8.4,8' x 4',20.79,6:01pm,10:02pm,2:03am,34°
3,M 42,Orion Nebula,"NGC 1976, Sh 2-281, LBN 974",Diffuse nebula,Orion,"05h 35' 18""","-05° 22' 60""",4.0,1.5° x 1°,21.96,4:38pm,10:03pm,3:28am,62°
4,M 43,De Mairan's Nebula,NGC 1982,Bright nebula,Orion,"05h 35' 30""","-05° 16' 00""",7.0,20' x 15',21.82,4:38pm,10:03pm,3:28am,62°


## Session parameters

Sets up the observation session (location, and my arrival time in the evening of the 3rd). Calculates the start and end of astronomical twilight, and notes when midnight is.

In [4]:
tammin = Observer(
    location=EarthLocation(
        lat='-31° 38\' 21.6"',
        lon='117° 29\' 05.8"',
        height=311 * u.m
    ),
    name='Tammin',
    timezone='Australia/Perth'
)

arrival_time = parse_to_Time('2019-01-03T07:00:00.000+08:00')
observation_start = tammin.twilight_evening_astronomical(
    arrival_time,
    which='next'
)
observation_end = tammin.twilight_morning_astronomical(
    arrival_time,
    which='next'
)
midnight = tammin.midnight(arrival_time, which='next')

## Rise, set and transit times for targets

Uses astropy and astroplan to calculate the rise, set, and transit times for the targets. The first few rise times are printed.

In [5]:
# Prevents various warnings from being emitted, described below
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # Warns about some hours-minute-seconds having 60 seconds
    coordinates = SkyCoord(data['Right Ascension'], data['Declination'])
    # Warns that some objects never rise
    rise_times = tammin.target_rise_time(
        observation_end,
        coordinates,
        which='previous'
    )
    # Warns that some objects never set
    set_times = tammin.target_set_time(
        observation_start,
        coordinates,
        which='next'
    )
    transit_times = tammin.target_meridian_transit_time(
        observation_end,
        coordinates,
        which='previous'
    )

print(rise_times[:5])

[2458486.81888519 2458486.94087414 2458486.91147209 2458486.86268526
 2458486.86302571]


## Format the data into a table so the template accepts it

Takes the target details and formats them nicely into a table that the HTML template can use to render the observing sheet. Comments below describe the least obvious columns.

In [6]:
output_data = subset_and_rename(data, {
    'Catalogue Entry': 'CatalogueEntry',
    'Familiar Name': 'FamiliarName',
    'Alternative Entries': 'AlternativeNames',
    'Type': 'Type',
    'Constellation': 'Constellation',
    'Magnitude': 'Magnitude'
})

output_data['RightAscension'] = coordinates.ra.astype(float)
output_data['RightAscensionString'] = coordinates.ra.to_string(u.h, pad=True)
output_data['Declination'] = coordinates.dec.astype(float)
output_data['DeclinationString'] = coordinates.dec.to_string(alwayssign=True)
output_data['RiseTime'] = safe_astropy_time_to_datetime(rise_times, tammin)
output_data['TransitTime'] = tammin.astropy_time_to_datetime(transit_times)
output_data['SetTime'] = safe_astropy_time_to_datetime(set_times, tammin)
# Altitude of each object at the start of the session
output_data['StartAltitude'] = altitude_at(
    coordinates,
    observation_start,
    tammin
).astype(float)
# Altitude of each object at midnight
output_data['MidnightAltitude'] = altitude_at(
    coordinates,
    midnight,
    tammin
).astype(float)
# Altitude of the object at the transit time
output_data['TransitAltitude'] = altitude_at(
    coordinates,
    transit_times,
    tammin
).astype(float)
# Altitude of each object at the end of the session
output_data['EndAltitude'] = altitude_at(
    coordinates,
    observation_end,
    tammin
).astype(float)

# Clean up the size of the object for formatting
sizes_split = data['Size'].str.split(' x ')
output_data['LongSize'] = Angle(sizes_split.str.get(0))
output_data['ShortSize'] = Angle([
    size[1] if len(size) == 2 else size[0] for size in sizes_split
])

# Chart on which to find the object the the Pocket sky atlas
output_data['ChartNumber'] = [
    map_number(coordinate) for coordinate in coordinates
]

# Sort the values by SetTime and TransitTime (the latter for those objects
# that never set)
output_data.sort_values(by=['SetTime', 'TransitTime'], inplace=True)

output_data.head(5)

,CatalogueEntry,FamiliarName,AlternativeNames,Type,Constellation,Magnitude,RightAscension,RightAscensionString,Declination,DeclinationString,RiseTime,TransitTime,SetTime,StartAltitude,MidnightAltitude,TransitAltitude,EndAltitude,LongSize,ShortSize,ChartNumber
43,M 33,Triangulum Galaxy,"NGC 598, CGCG 502-110, MCG 5-4-69, PGC 5818, U...",Galaxy,Triangulum,5.7,23.466667,01h33m52s,30.658056,+30d39m29s,2019-01-03 14:20:41.249514+08:00,2019-01-03 18:53:53.478646+08:00,2019-01-03 23:27:05.186679+08:00,21.033360,-8.403188,27.605178,-47.430366,66.0,41.6,1 to 10
44,M 74,,"NGC 628, CGCG 460-14, IRAS 01340+1532, MCG 3-5...",Galaxy,Pisces,9.4,24.175000,01h36m42s,15.783333,+15d47m00s,2019-01-03 13:37:55.317103+08:00,2019-01-03 18:56:39.981625+08:00,2019-01-04 00:15:24.359205+08:00,34.298120,0.168484,42.481831,-41.291477,10.5,9.5,1 to 10
45,M 77,,"NGC 1068, 3C 71, Arp 37, CGCG 388-98, IRAS 024...",Galaxy,Cetus,8.9,40.670833,02h42m41s,-0.012778,-0d00m46s,2019-01-03 14:03:33.991275+08:00,2019-01-03 20:02:25.511582+08:00,2019-01-04 02:01:16.942250+08:00,55.704964,22.526674,58.295887,-19.139754,7.1,6.0,1 to 10
46,M 45,Pleiades,Mel 22,Bright nebula,Taurus,1.2,56.750000,03h47m00s,24.116667,+24d07m00s,2019-01-03 16:11:46.344603+08:00,2019-01-03 21:06:43.732659+08:00,2019-01-04 02:01:41.088448+08:00,34.143395,18.299992,34.186944,-17.612050,102.0,102.0,11 to 20
1,M 38,,"NGC 1912, OCL 433",Open cluster,Auriga,6.4,82.179167,05h28m43s,35.855000,+35d51m18s,2019-01-03 18:34:51.525556+08:00,2019-01-03 22:48:19.119473+08:00,2019-01-04 03:01:46.837186+08:00,17.642927,19.511798,22.492310,-4.759822,15.0,15.0,11 to 20


## Render the targets into an observing sheet

Uses the template in the `astromateur` folder to render the observing sheet, and saves it to [targets.html](targets.html).

In [7]:
rendered_html = template.render(
    targets=output_data,
    start_time=tammin.astropy_time_to_datetime(observation_start),
    end_time=tammin.astropy_time_to_datetime(observation_end),
    ordered_by='set time, then transit time'
)

with open('targets.html', 'w') as file:
    file.write(rendered_html)